In [31]:
import pandas as pd
df = pd.read_csv("final.csv", low_memory=False)
df.head()

,Sl no,Name,Profile ID,Comment,Comment_Tokens,comment_length,word_count,sentiment_score,sentiment_class
0,1,scotty2jatty,398274122,you better do dirty,"better,dirty",19,4,0,neutral
1,2,angelasanders975,65963290409,thats soooooooooo right,"thats,soooooooooo,right",23,3,0,neutral
2,3,myleslewis_24,11687291106,bro i want that shirt,"bro,want,shirt",21,5,1,positive
3,4,miladheydarpour1369,7529960718,good,good,4,1,3,positive
4,5,matthew_williams224,25240207448,bro he only needs to beat jakes ass,"bro,needs,beat,jakes,ass",35,8,-7,negative


In [32]:
# Define a mapping dictionary for sentiment classes
sentiment_mapping = {
    'neutral': 1,
    'positive': 1,
    'negative': 0
}

# Apply the mapping to the 'sentiment_class' column to create a numerical 'sentiment_label'
df['sentiment_label'] = df['sentiment_class'].map(sentiment_mapping)

# Display the first few rows to confirm
print(df[['sentiment_label', 'sentiment_class']].head())

   sentiment_label sentiment_class
0                1         neutral
1                1         neutral
2                1        positive
3                1        positive
4                0        negative


In [33]:
df.head()

,Sl no,Name,Profile ID,Comment,Comment_Tokens,comment_length,word_count,sentiment_score,sentiment_class,sentiment_label
0,1,scotty2jatty,398274122,you better do dirty,"better,dirty",19,4,0,neutral,1
1,2,angelasanders975,65963290409,thats soooooooooo right,"thats,soooooooooo,right",23,3,0,neutral,1
2,3,myleslewis_24,11687291106,bro i want that shirt,"bro,want,shirt",21,5,1,positive,1
3,4,miladheydarpour1369,7529960718,good,good,4,1,3,positive,1
4,5,matthew_williams224,25240207448,bro he only needs to beat jakes ass,"bro,needs,beat,jakes,ass",35,8,-7,negative,0


In [34]:
# General libraries
import numpy as np
import pandas as pd
import random as rn

# Neural network libraries
import tensorflow as tf  # TensorFlow backend
from tensorflow.keras.layers import Input  # For input layer
from tensorflow.keras.layers import Dropout  # For random dropout
from tensorflow.keras.layers import Embedding  # For embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D  # For convolution layer
from tensorflow.keras.layers import concatenate  # For concatenation
from tensorflow.keras.layers import Activation  # For activation layer
from tensorflow.keras.layers import Dense  # For fully connected layer
from tensorflow.keras.models import Model  # Model groups layers into an object with training and inference features.

# Libraries for data formatting
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Word embedding loading library
from gensim.models import KeyedVectors

# Suppress TensorFlow info logs
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# To ensure TensorFlow uses only the CPU
tf.config.set_visible_devices([], 'GPU')

print("Using TensorFlow version:", tf.__version__)


Using TensorFlow version: 2.18.0


In [35]:
SEED = 123
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [36]:
# Splitting into train and test sets (80/20 split)
df_train = df.sample(frac=0.8, random_state=123)  # 80% of the data
df_test = df.drop(df_train.index)  # Remaining 20% of the data

# Selecting only the 'Comment' and 'sentiment_class' columns
df_train = df_train[["Comment", "sentiment_label"]]
df_test = df_test[["Comment", "sentiment_label"]]

df_train, df_test

(                                                Comment  sentiment_label
 4933  you said good bye dumb bitch snd i said okwhat...                0
 184                                        amen to that                1
 2399  tevin is my favorite sophomore i think hes so ...                1
 1520          trump means king of truth and pragmatisme                1
 3519                             who that pussy wet for                0
 ...                                                 ...              ...
 1924                                               good                1
 806   he stands for diversity equity inclusion and t...                0
 1808                                               good                1
 1076                                       debate trump                1
 1261  im disgusted with the people who still vote fo...                0
 
 [4071 rows x 2 columns],
                                                 Comment  sentiment_label
 2        

In [37]:
df_train.groupby('sentiment_label').count()

,Comment
sentiment_label,
0,1909
1,2162


In [38]:
len(df_train)

4071

In [39]:
# separate independent variabe (text) and dependednt variable (label)
X_train, y_train = df_train['Comment'].values, df_train['sentiment_label'].values

In [40]:

X_train[:5]

array(['you said good bye dumb bitch snd i said okwhat is it with you white devilsi have nothing for you vampire lover of blood',
       'amen to that',
       'tevin is my favorite sophomore i think hes so funny hes really not lmaoo hes a little hoe',
       'trump means king of truth and pragmatisme',
       'who that pussy wet for'], dtype=object)

In [41]:
# check what we got in dependent variabl
y_train[:5]

array([0, 1, 1, 1, 0])

In [42]:
df_test.head()

,Comment,sentiment_label
2,bro i want that shirt,1
3,good,1
8,yeah boy tysons gonna get shit done the goat,0
14,i cant wait to see it,1
16,you do have a money plan get punched in the mo...,1


In [43]:
# seperate variabes of test data
X_test, y_test = df_test['Comment'].values, df_test['sentiment_label'].values

In [44]:
num_words = 100000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)
xtrain = tokenizer.texts_to_sequences(X_train) # coverts text to numbers
print(xtrain[:2])

[[3, 95, 11, 703, 195, 7, 3023, 5, 95, 3024, 8, 16, 23, 3, 140, 3025, 28, 212, 13, 3, 3026, 1949, 10, 985], [1144, 4, 9]]


In [45]:
maxlen = max(map(lambda x: len(x), xtrain))
xtrain = pad_sequences(xtrain, maxlen=maxlen) # padding each row to make them same length
print(xtrain[:2])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [68]:
print(maxlen)

393


In [46]:
# convert the text of test data
xtest = tokenizer.texts_to_sequences(X_test) # coverts text to numbers
xtest = pad_sequences(xtest, maxlen=maxlen) 

In [47]:
from gensim.scripts.glove2word2vec import glove2word2vec


glove_file = 'glove.6B.50d.txt'  
word2vec_file = 'glove.6B.50d_w2v.txt'  


import os
if not os.path.exists(glove_file):
    print(f"Error: GloVe file not found at {glove_file}")
else:
    # Convert GloVe format to Word2Vec format
    try:
        glove2word2vec(glove_file, word2vec_file)
        print(f"Successfully converted GloVe file to Word2Vec format: {word2vec_file}")
    except Exception as e:
        print(f"Error during conversion: {e}")

/tmp/ipykernel_3406/2388528531.py:14: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_file)


Successfully converted GloVe file to Word2Vec format: glove.6B.50d_w2v.txt


In [48]:
from gensim.models import KeyedVectors

# Load the Word2Vec model
word2vec_model = KeyedVectors.load_word2vec_format('glove.6B.50d_w2v.txt', binary=False)

# Check the model to verify it's loaded properly
print("Word2Vec model loaded successfully!")


Word2Vec model loaded successfully!


In [52]:
import numpy as np

embedding_dim = 50  # Dimension of the embeddings (corresponding to 'glove.6B.50d.txt')

# Assuming you have a tokenizer and it is already fitted
num_words = min(10000, len(tokenizer.word_index))  # Limit the size of the vocabulary

embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = word2vec_model.get_vector(word) if word in word2vec_model else None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(f"Embedding matrix shape: {embedding_matrix.shape}")


Embedding matrix shape: (7458, 50)


In [53]:
xtrain

array([[   0,    0,    0, ..., 1949,   10,  985],
       [   0,    0,    0, ..., 1144,    4,    9],
       [   0,    0,    0, ...,    2,  169,   41],
       ...,
       [   0,    0,    0, ...,    0,    0,   11],
       [   0,    0,    0, ...,    0,  575,   44],
       [   0,    0,    0, ...,  148,   13,   94]], dtype=int32)

In [54]:
xtrain_clipped = np.clip(xtrain, 0, num_words - 1)

In [55]:
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Input, concatenate, Activation
from keras.models import Model

def create_cnn_model():
    comment_input = Input(shape=(maxlen,), dtype='int32')
    
    print('loading word vectors')
    # Use the pre-trained embedding matrix for the Embedding layer
    comment_encoder = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True)(comment_input)
    comment_encoder = Dropout(0.5)(comment_encoder)
    
    # Bigram branch
    bigram_branch = Conv1D(filters=128, kernel_size=3, padding='valid', activation='relu', strides=1)(comment_encoder)
    bigram_branch = GlobalMaxPooling1D()(bigram_branch)
    bigram_branch = Dropout(0.5)(bigram_branch)
    
    # Trigram branch
    trigram_branch = Conv1D(filters=256, kernel_size=4, padding='valid', activation='relu', strides=1)(comment_encoder)
    trigram_branch = GlobalMaxPooling1D()(trigram_branch)
    trigram_branch = Dropout(0.2)(trigram_branch)
    
    # Fourgram branch
    fourgram_branch = Conv1D(filters=512, kernel_size=5, padding='valid', activation='relu', strides=1)(comment_encoder)
    fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
    fourgram_branch = Dropout(0.2)(fourgram_branch)
    
    # Merging the branches
    merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

    # Fully connected layer
    merged = Dense(256, activation='relu')(merged)
    merged = Dropout(0.5)(merged)
    
    # Output layer
    merged = Dense(1)(merged)
    output = Activation('sigmoid')(merged)
    
    # Create and compile the model
    model = Model(inputs=[comment_input], outputs=[output])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

# Assuming `xtrain`, `y_train`, `embedding_matrix`, and other required variables are already defined
cnn_model = create_cnn_model()
cnn_model.fit(xtrain_clipped, y_train, epochs=3, batch_size=32, verbose=1)


/anaconda/envs/azureml_py38/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_17']
Received: inputs=Tensor(shape=(None, 393))
  warnings.warn(msg)


128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.6063 - loss: 0.7495
Epoch 2/3
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.7541 - loss: 0.4898
Epoch 3/3
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.8051 - loss: 0.4226


In [56]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report
import numpy as np
import datetime

def evaluate_binary_classification(gnd_truths, predictions, model_name='', path=''):
    # Convert to numpy arrays if not already
    gnd_truths = np.array(gnd_truths)
    predictions = np.array(predictions)
    
    # Calculate basic metrics
    tp = np.count_nonzero(predictions * gnd_truths)
    tn = np.count_nonzero((predictions - 1) * (gnd_truths - 1))
    fp = np.count_nonzero(predictions * (gnd_truths - 1))
    fn = np.count_nonzero((predictions - 1) * gnd_truths)
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1measure = (2 * precision * recall) / (precision + recall)
    
    cks = cohen_kappa_score(predictions, gnd_truths)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(gnd_truths, predictions)
    auc_ = auc(false_positive_rate, true_positive_rate)
    
    # Basic Report
    basic_report = {
        'Total Samples': len(gnd_truths),
        'Positive Samples': sum(gnd_truths),
        'Negative Samples': len(gnd_truths) - sum(gnd_truths),
        'True Positive': tp,
        'True Negative': tn,
        'False Positive': fp,
        'False Negative': fn,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Measure': f1measure,
        'Cohen Kappa Score': cks,
        'Area Under Curve': auc_
    }
    
    # Full Report
    full_report = get_str(basic_report) + '\n'
    full_report += classification_report(gnd_truths, predictions)
    
    # Save Reports if path is provided
    if path:
        save_report(basic_report, 'basic_report.txt', path)
        save_report(full_report, 'full_report.txt', path)
    
    return basic_report, full_report


def get_str(report):
    """Helper function to convert a report dictionary to string format."""
    str_ = ''
    for key in report.keys():
        str_ += f'{key}\t{report[key]}\n'
    return str_


def save_report(report, filename, path):
    """Helper function to save the report to a file."""
    with open(path + filename, 'a+') as FO:
        FO.write(report)

# Example usage
# You will provide gnd_truths and predictions here
# gnd_truths = [0, 1, 1, 0, 1]
# predictions = [0, 1, 0, 0, 1]

# basic_report, full_report = evaluate_binary_classification(gnd_truths, predictions)

# Printing the reports
# print("Basic Report:")
# print(basic_report)

# print("\nFull Report:")
# print(full_report)


In [57]:
# Assuming cnn_model, xtest, and y_test are already defined
p = cnn_model.predict(xtest, verbose=1)
predicted = [int(round(x[0])) for x in p]  # Convert the predictions to 0 or 1
actual = y_test  # Actual ground truth labels

# Call the evaluate_binary_classification function
basic_report, full_report = evaluate_binary_classification(gnd_truths=actual, predictions=predicted)

# Print the full report
print(full_report)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Total Samples	1018
Positive Samples	522
Negative Samples	496
True Positive	437
True Negative	413
False Positive	83
False Negative	85
Accuracy	0.8349705304518664
Precision	0.8403846153846154
Recall	0.8371647509578544
F1 Measure	0.8387715930902112
Cohen Kappa Score	0.6697587839560049
Area Under Curve	0.8349130206402174

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       496
           1       0.84      0.84      0.84       522

    accuracy                           0.83      1018
   macro avg       0.83      0.83      0.83      1018
weighted avg       0.83      0.83      0.83      1018



In [64]:
cnn_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 393)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 393, 50)   │    372,900 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 393, 50)   │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 391, 128)  │     19,328 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 390, 256)  │     51,456 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 389, 512)  │    128,512 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_3[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 256)       │          0 │ conv1d_4[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ conv1d_5[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 128)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 256)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 512)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 896)       │          0 │ dropout_6[0][0],  │
│ (Concatenate)       │                   │            │ dropout_7[0][0],  │
│                     │                   │            │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    229,632 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        257 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 1)         │          0 │ dense_3[0][0]     │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,406,257 (9.18 MB)

 Trainable params: 802,085 (3.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,604,172 (6.12 MB)

In [69]:
def predict_sentiment(model, tokenizer, text_input, maxlen):
    """
    Predict sentiment for a single text input or a batch of text inputs.
    
    Args:
    - model: The trained model (CNN in your case)
    - tokenizer: The tokenizer used to preprocess text
    - text_input: A single text string or a list of text strings
    - maxlen: Maximum length of input sequences (padding length)
    
    Returns:
    - predictions: A list of predicted sentiment labels (0 for negative, 1 for positive)
    """
    print(f"Received input text: {text_input}")  # Debugging line
    
    if isinstance(text_input, str):
        # If a single text string, make it a list
        text_input = [text_input]

    # Convert the input texts to sequences
    text_sequences = tokenizer.texts_to_sequences(text_input)
    print(f"Text sequences after tokenization: {text_sequences}")  # Debugging line
    
    # Pad the sequences to ensure uniform length
    text_padded = pad_sequences(text_sequences, maxlen=maxlen)
    print(f"Padded sequences shape: {text_padded.shape}")  # Debugging line
    
    # Make predictions
    try:
        predictions = model.predict(text_padded)
        print(f"Model predictions: {predictions}")  # Debugging line
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None

    # Convert probabilities to binary sentiment labels (0 or 1)
    sentiment_labels = (predictions > 0.5).astype(int)
    print(f"Predicted sentiment labels: {sentiment_labels}")  # Debugging line
    
    return sentiment_labels.flatten()

# User input loop for continuous prediction
def user_input_loop(model, tokenizer, maxlen):
    print("Starting prediction loop...")
    while True:
        # Ask the user to input a comment
        input_text = input("Enter a comment to predict sentiment (or 'exit' to quit): ")
        
        if input_text.lower() == 'exit':
            print("Exiting prediction loop.")
            break
        
        # Make prediction for the entered text
        sentiment = predict_sentiment(model, tokenizer, input_text, maxlen)
        
        # Check if prediction was successful
        if sentiment is not None:
            sentiment_label = "Positive" if sentiment[0] == 1 else "Negative"
            # Output the result
            print(f"The sentiment of the comment is: {sentiment_label}")
        else:
            print("Prediction failed.")
        
        print("-" * 50)

# Debugging Step: Check if the model is loaded properly
print("Loading model...")
# Assuming cnn_model and tokenizer are already loaded or created in the script
print("Model loaded:", cnn_model)
print("Tokenizer loaded:", tokenizer)

# Example usage
# Assuming cnn_model, tokenizer, and maxlen are already defined
user_input_loop(cnn_model, tokenizer, 393)  # Update maxlen to 393


Loading model...
Model loaded: <Functional name=functional_1, built=True>
Tokenizer loaded: <keras.src.legacy.preprocessing.text.Tokenizer object at 0x7efbcf0a2a70>
Starting prediction loop...
Received input text: nigga
Text sequences after tokenization: [[72]]
Padded sequences shape: (1, 393)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Model predictions: [[0.4261403]]
Predicted sentiment labels: [[0]]
The sentiment of the comment is: Negative
--------------------------------------------------
Exiting prediction loop.


In [70]:
cnn_model.save('CNN_NEW.h5')

In [71]:
import pickle

# Save the tokenizer to a pickle file
with open('CNN_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)